In [1]:
import pandas as pd
import numpy as np

In [2]:
# data separation 
d = pd.read_csv("match_team_player.csv")
match_player = d.iloc[:, 106:]
#match_player.insert(0, "match_id", d.match_id)
#match_team.to_csv("match_team.csv", index = False)
match_player.to_csv("match_player.csv", index = False)

In [3]:
d.columns[106] # player starts from 106

'home_overall_rating_1'

In [4]:
# clean players
columns_to_drop = ['preferred', 'work_rate', "birthday", 'player_name',  'weight', 'height'] # delete ordinals
data = match_player
names = data.columns

def ifdrop(name, columns):
    for c in columns:
        if name.find(c) != -1:
            return True
    return False

to_drop = [x for x in names if ifdrop(x, columns_to_drop)]
print(len(to_drop)/22)
data.drop(columns = to_drop, inplace = True) # Now each player has 35 attributes
names = data.columns

7.0


In [5]:
data.shape

(25979, 770)

In [151]:
# OLD VERSION: sort players
def row_combine_and_sort(row):
    '''
    New row will have (11-4)*2*2=28 fewer entries. From 836 to 808
    Attributes include:  36*11*2=814 (numerics), 2*4*2=16(ordinals)
    '''
    names = row.index
    num_features = 38
    ord_locs = [i for i,x in enumerate(names[:num_features]) if x.find("work_rate") != -1]
    
    
    def attribute_sort(start):
        index = [start + i * num_features for i in range(22)]
        home_values, away_values = row[index[:11]], row[index[11:]]
        home_values, away_values = home_values.to_list(), away_values.to_list()
        home_values.sort(reverse=True)
        away_values.sort(reverse=True)
        return home_values + away_values
    
    def ord_summary(nums): # skipped now, we don't want to keep any ordinal attribute anymore
        highs = sum([x=='high' for x in nums])
        mediums = sum([x=='medium' for x in nums])
        lows = sum([x=='low' for x in nums])
        missings = sum([x != x for x in nums])
        return [highs, mediums, lows, missings]
    
    def attribute_ord(start):
        index = [start + i * num_features for i in range(22)]
        home_values, away_values = row[index[:11]].to_list(), row[index[11:]].to_list()
        home_info = ord_summary(home_values)
        away_info = ord_summary(away_values)
        return home_info + away_info
    
    newrow = []
    row_names = []
    for i in range(num_features):
        if i not in ord_locs:
            newrow.extend(attribute_sort(i))
            col = names[i][:-1][4:]
            row_names.extend(['home' + col + str(i) for i in range(1,12,1)])
            row_names.extend(['away' + col + str(i) for i in range(1,12,1)])
        else:
            newrow.extend(attribute_ord(i))
            col = names[i][:-1][4:]
            row_names.extend(['home' + col + x for x in ['highs', 'mediums', 'lows', 'missings']])
            row_names.extend(['away' + col + x for x in ['highs', 'mediums', 'lows', 'missings']])
    
    assert len(newrow) == len(row_names)
    return pd.Series(newrow, index = row_names)

In [6]:
# NEW VERSION: sort players
def row_combine_and_sort(row):
    names = row.index
    num_features = 35

    def attribute_sort(start):
        index = [start + i * num_features for i in range(22)]
        home_values, away_values = row[index[:11]], row[index[11:]]
        home_values, away_values = home_values.to_list(), away_values.to_list()
        home_values.sort(reverse=True)
        away_values.sort(reverse=True)
        return home_values + away_values
    
    newrow = []
    row_names = []
    for i in range(num_features):
        newrow.extend(attribute_sort(i))
        col = names[i][:-1][4:]
        row_names.extend(['home' + col + str(i) for i in range(1,12,1)])
        row_names.extend(['away' + col + str(i) for i in range(1,12,1)])
    
    assert len(newrow) == len(row_names)
    return pd.Series(newrow, index = row_names)

In [7]:
newrows = [row_combine_and_sort(row) for _,row in data.iterrows()]
newdata = pd.concat(newrows, axis = 1)
newdata = newdata.transpose()
data_players = newdata

In [8]:
data_players.shape # it should have 770 numerical columns

(25979, 770)

In [9]:
data_players.to_csv("match_player_cleaned.csv", index = False)

In [11]:
# keep complete data
data_team = pd.read_csv("final_match_team.csv").drop(columns = ["season", "country_name", "league_name"])
print('data team  shape', data_team.shape)
data_all = pd.concat([data_team.drop(columns = ["outcome"]), data_players], axis=1)
print('data all  shape', data_all.shape)

data team  shape (25979, 29)
data all  shape (25979, 798)


In [12]:
rows_complete = np.mean(np.isnan(np.array(data_all)), axis = 1) == 0
print("complete rows", np.sum(rows_complete))
data_all = pd.concat([data_team, data_players], axis=1)
data_complete = data_all.iloc[rows_complete,:]
data_complete.shape

complete rows 16686


(16686, 799)

In [13]:
data_complete.to_csv("match_team_player_complete.csv", index = False)

All things below correspond to some ad-hoc attempts to reduce and summarize the players features. They are not maintained anymore.

In [60]:
newrows = [row_summary_stats(row) for _,row in data_later_complete.iterrows()]
newdata = pd.concat(newrows, axis = 1)
newdata = newdata.transpose()
newdata.shape

(16686, 253)

In [61]:
newdata.columns[29:37]

Index(['home_attacking_work_rate_highs', 'home_attacking_work_rate_mediums',
       'away_attacking_work_rate_highs', 'away_attacking_work_rate_mediums',
       'home_defensive_work_rate_highs', 'home_defensive_work_rate_mediums',
       'away_defensive_work_rate_highs', 'away_defensive_work_rate_mediums'],
      dtype='object')